In [ ]:
# https://github.com/rainym00d/LLM4RS/tree/main 
# data preprocess: https://github.com/rainym00d/LLM4RS/blob/main/data/preprocess/News.ipynb

In [4]:
import openai 
import pandas as pd
import numpy as np
import warnings
import time
import itertools
import copy
warnings.filterwarnings("ignore")
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth',100)

In [5]:
MIND_dev_news_df = pd.read_csv("MIND/news_test.tsv", header=None, delimiter="\t")
MIND_dev_news_df.columns = ["newsid","category","subcategory","title","abstract","url","title_entities","abstract_entities"]
MIND_dev_news_df.drop(['url','title_entities','abstract_entities'], axis=1, inplace=True)
MIND_dev_news_df
(MIND_dev_news_df.isna().sum()/MIND_dev_news_df.shape[0]).sort_values(ascending=False)

,newsid,category,subcategory,title,abstract
0,N55528,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, and Prince Philip Swear By","Shop the notebooks, jackets, and more that the royals can't live without."
1,N19639,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding you back and keeping you from shedding that unwanted...
2,N61837,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches of Ukraine's War,Lt. Ivan Molchanets peeked over a parapet of sand bags at the front line of the war in Ukraine. ...
3,N53526,health,voices,I Was An NBA Wife. Here's How It Affected My Mental Health.,"I felt like I was a fraud, and being an NBA wife didn't help that. In fact, it nearly destroyed me."
4,N38324,health,medical,"How to Get Rid of Skin Tags, According to a Dermatologist","They seem harmless, but there's a very good reason you shouldn't ignore them. The post How to Ge..."
...,...,...,...,...,...
72,N16909,weather,weathertopstories,"Adapting, Learning And Soul Searching: Reflecting On Woolsey Fire","Woolsey Fire Anniversary: A community is forever changed by the largest blaze in county history,..."
73,N47585,lifestyle,lifestylefamily,Family says 13-year-old Broadway star died from massive asthma attack,NaN
74,N7482,sports,more_sports,St. Dominic soccer player tries to kick cancer to the curb,"Sometimes, what happens on the sidelines can be even more important than what happens on the fie..."
75,N34418,sports,soccer_epl,How the Sounders won MLS Cup,"Mark, Jeremiah and Casey were so excited they did a postgame podcast."


abstract       0.090909
newsid         0.000000
category       0.000000
subcategory    0.000000
title          0.000000
dtype: float64

In [6]:
MIND_dev_news_df_fliter = MIND_dev_news_df.dropna().reset_index(drop=True)
MIND_dev_news_df_fliter

,newsid,category,subcategory,title,abstract
0,N55528,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, and Prince Philip Swear By","Shop the notebooks, jackets, and more that the royals can't live without."
1,N19639,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding you back and keeping you from shedding that unwanted...
2,N61837,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches of Ukraine's War,Lt. Ivan Molchanets peeked over a parapet of sand bags at the front line of the war in Ukraine. ...
3,N53526,health,voices,I Was An NBA Wife. Here's How It Affected My Mental Health.,"I felt like I was a fraud, and being an NBA wife didn't help that. In fact, it nearly destroyed me."
4,N38324,health,medical,"How to Get Rid of Skin Tags, According to a Dermatologist","They seem harmless, but there's a very good reason you shouldn't ignore them. The post How to Ge..."
...,...,...,...,...,...
65,N17258,news,newsscienceandtechnology,Realme takes chunk of India mobile market as Samsung slides,Over 400 percent more phones shipped year-on-year
66,N23858,sports,golf,Young Northeast Florida fans flock to U.S. women's soccer,When the U.S. women's national soccer team arrived in Jacksonville for a final exhibition match ...
67,N16909,weather,weathertopstories,"Adapting, Learning And Soul Searching: Reflecting On Woolsey Fire","Woolsey Fire Anniversary: A community is forever changed by the largest blaze in county history,..."
68,N7482,sports,more_sports,St. Dominic soccer player tries to kick cancer to the curb,"Sometimes, what happens on the sidelines can be even more important than what happens on the fie..."


In [7]:
drop_newsid_list = pd.concat([MIND_dev_news_df, MIND_dev_news_df_fliter]).drop_duplicates(["newsid"],keep=False).newsid.values.tolist()
drop_newsid_list

['N3354', 'N56061', 'N56193', 'N10529', 'N55199', 'N47585', 'N44276']

In [8]:
MIND_dev_df = pd.read_csv("MIND/behaviors.tsv", header=None, delimiter="\t")
MIND_dev_df.columns = ["impressionid","userid","time","history","impression"]
MIND_dev_df["history"] = MIND_dev_df["history"].apply(lambda x: str(x).split(' '))
MIND_dev_df["impression"] = MIND_dev_df["impression"].apply(lambda x: str(x).split(' '))
MIND_dev_df

,impressionid,userid,time,history,impression
0,1,U13740,11/11/2019 9:05:58 AM,"[N55189, N42782, N34694, N45794, N18445, N63302, N10414, N19347, N31801]","[N55689-1, N35729-0]"
1,2,U91836,11/12/2019 6:11:30 PM,"[N31739, N6072, N63045, N23979, N35656, N43353, N8129, N1569, N17686, N13008, N21623, N6233, N14...","[N20678-0, N39317-0, N58114-0, N20495-0, N42977-0, N22407-0, N14592-0, N17059-1, N33677-0, N7821..."
2,3,U73700,11/14/2019 7:01:48 AM,"[N10732, N25792, N7563, N21087, N41087, N5445, N60384, N46616, N52500, N33164, N47289, N24233, N...","[N50014-0, N23877-0, N35389-0, N49712-0, N16844-0, N59685-0, N23814-1, N23446-0, N64174-0, N1181..."
3,4,U34670,11/11/2019 5:28:05 AM,"[N45729, N2203, N871, N53880, N41375, N43142, N33013, N29757, N31825, N51891]","[N35729-0, N33632-0, N49685-1, N27581-0]"
4,5,U8125,11/12/2019 4:11:21 PM,"[N10078, N56514, N14904, N33740]","[N39985-0, N36050-0, N16096-0, N8400-1, N22407-0, N60408-0, N61497-0, N47412-0, N41220-0, N1940-..."
...,...,...,...,...,...
156960,156961,U21593,11/14/2019 10:24:05 PM,"[N7432, N58559, N1954, N43353, N14343, N13008, N28833, N25114, N28833, N44495, N996, N14441, N61...","[N2235-0, N22975-0, N64037-0, N47652-0, N11378-0, N42515-0, N56211-0, N37204-0, N6837-0, N11930-..."
156961,156962,U10123,11/13/2019 6:57:04 AM,"[N9803, N104, N24462, N57318, N55743, N40526, N31726, N54842, N26026, N56889, N2309, N41298, N63...","[N3841-0, N61571-0, N58813-0, N28213-0, N4428-0, N25796-0, N13579-0, N32118-0, N22457-0, N62128-..."
156962,156963,U75630,11/14/2019 10:58:13 AM,"[N29898, N59704, N4408, N9803, N53644, N26103, N812, N60919, N49319, N54655, N41375, N31248, N52...","[N55913-0, N62318-0, N53515-0, N10960-0, N9135-0, N57081-0, N8595-0, N30648-0, N47572-0, N20676-..."
156963,156964,U44625,11/13/2019 2:57:02 PM,"[N4118, N47297, N3164, N43295, N6056, N38747, N42973, N25113, N53854, N44649, N41526, N47754, N1...","[N6219-0, N3663-0, N31147-0, N58363-0, N4107-0, N4573-0, N42815-0, N27515-0, N20495-0, N18571-0,..."


In [9]:
def get_clean_impression_and_history(row):
    impression_list = []
    impression_pos_list = []
    label_list = []
    flag = 0
    for i in row.impression:
        impression, label = i.split("-")
        if impression in drop_newsid_list:
            continue
        label = int(label)
        if label == 1:
            impression_pos_list.append(impression)
            if flag == 1:
                continue
            else:
                impression_list.append(impression)
                label_list.append(label)
                flag=1
        else:
            impression_list.append(impression)
            label_list.append(label)
    
    if 1 not in label_list:
        pos_target = -1
    else:
        pos_target = impression_list[label_list.index(1)]
        
    history_list = copy.deepcopy(row.history)
    for i in row.history:
        if i in drop_newsid_list:
            history_list.remove(i)
     
    return [impression_list, label_list, pos_target, history_list, impression_pos_list]

In [10]:
MIND_dev_df[["impression_list", "label_list", "pos_target", "history_list", "impression_pos_list"]] = MIND_dev_df.apply(get_clean_impression_and_history, axis=1, result_type="expand")
MIND_dev_df.drop(['time','impression','history'], axis=1, inplace=True)
MIND_dev_df

,impressionid,userid,impression_list,label_list,pos_target,history_list,impression_pos_list
0,1,U13740,"[N55689, N35729]","[1, 0]",N55689,"[N55189, N42782, N34694, N45794, N18445, N63302, N10414, N19347, N31801]",[N55689]
1,2,U91836,"[N20678, N39317, N58114, N20495, N42977, N22407, N14592, N17059, N33677, N7821, N6890]","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]",N17059,"[N31739, N6072, N63045, N23979, N35656, N43353, N8129, N1569, N17686, N13008, N21623, N6233, N14...",[N17059]
2,3,U73700,"[N50014, N23877, N35389, N49712, N16844, N59685, N23814, N23446, N64174, N11817, N60550, N48225,...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",N23814,"[N10732, N25792, N7563, N21087, N41087, N5445, N60384, N46616, N52500, N33164, N47289, N24233, N...",[N23814]
3,4,U34670,"[N35729, N33632, N49685, N27581]","[0, 0, 1, 0]",N49685,"[N45729, N2203, N871, N53880, N41375, N43142, N33013, N29757, N31825, N51891]",[N49685]
4,5,U8125,"[N39985, N36050, N16096, N8400, N22407, N60408, N61497, N47412, N41220, N1940, N724, N11363, N26...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",N8400,"[N10078, N56514, N14904, N33740]",[N8400]
...,...,...,...,...,...,...,...
156960,156961,U21593,"[N2235, N22975, N64037, N47652, N11378, N42515, N56211, N37204, N6837, N11930, N46917, N2960, N2...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]",N50055,"[N7432, N58559, N1954, N43353, N14343, N13008, N28833, N25114, N28833, N44495, N996, N14441, N61...",[N50055]
156961,156962,U10123,"[N3841, N61571, N58813, N28213, N4428, N25796, N13579, N32118, N22457, N62128, N21309, N59385, N...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",N30212,"[N9803, N104, N24462, N57318, N55743, N40526, N31726, N54842, N26026, N56889, N2309, N41298, N63...",[N30212]
156962,156963,U75630,"[N55913, N62318, N53515, N10960, N9135, N57081, N8595, N30648, N47572, N20676, N35011, N65027, N...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",N46283,"[N29898, N59704, N4408, N9803, N53644, N26103, N812, N60919, N49319, N54655, N41375, N31248, N52...",[N46283]
156963,156964,U44625,"[N6219, N3663, N31147, N58363, N4107, N4573, N42815, N27515, N20495, N18571, N4754, N63796, N120...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,...",N21712,"[N4118, N47297, N3164, N43295, N6056, N38747, N42973, N25113, N53854, N44649, N41526, N47754, N1...","[N21712, N50007, N366, N18573, N20630]"


In [11]:
impression_list = MIND_dev_df["impression_pos_list"].values.tolist()
history_list = MIND_dev_df["history_list"].values.tolist()
impression_list = list(itertools.chain(*impression_list))
history_list = list(itertools.chain(*history_list))
impression_list = pd.Series(impression_list + history_list)
item_pop = impression_list.value_counts().rename_axis("itemid").reset_index(name="pop")
item_pop

,itemid,pop
0,N306,20765
1,N42620,17918
2,N47020,16300
3,N31801,15694
4,N45794,15200
...,...,...
39857,N17563,1
39858,N14836,1
39859,N51579,1
39860,N49678,1


In [12]:
MIND_dev_df = MIND_dev_df[MIND_dev_df["pos_target"]!=-1].reset_index(drop=True)
MIND_dev_df["history_len"] = MIND_dev_df["history_list"].apply(lambda x: len(x))
MIND_dev_df["impression_len"] = MIND_dev_df["impression_list"].apply(lambda x: len(x))
MIND_dev_df_fliter = MIND_dev_df[(MIND_dev_df["history_len"]>=10) & (MIND_dev_df["impression_len"]>=10)].reset_index(drop=True)
MIND_dev_df_fliter

,impressionid,userid,impression_list,label_list,pos_target,history_list,impression_pos_list,history_len,impression_len
0,2,U91836,"[N20678, N39317, N58114, N20495, N42977, N22407, N14592, N17059, N33677, N7821, N6890]","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]",N17059,"[N31739, N6072, N63045, N23979, N35656, N43353, N8129, N1569, N17686, N13008, N21623, N6233, N14...",[N17059],82,11
1,3,U73700,"[N50014, N23877, N35389, N49712, N16844, N59685, N23814, N23446, N64174, N11817, N60550, N48225,...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",N23814,"[N10732, N25792, N7563, N21087, N41087, N5445, N60384, N46616, N52500, N33164, N47289, N24233, N...",[N23814],16,36
2,7,U8355,"[N51346, N33848, N15132, N10688, N6342, N61359, N7809, N64397, N27079, N47149, N39844, N52585, N...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",N55204,"[N8419, N15771, N1431, N5888, N18663, N24123, N22130, N20286, N32095, N46868, N55310, N31931, N3...",[N55204],35,173
3,11,U89744,"[N47572, N45523, N64560, N53245, N8509, N51951, N37185, N23814, N7901, N26220, N45590, N12028, N...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",N61622,"[N24422, N25287, N39121, N41777, N58226, N119, N29197, N33727, N45189, N13480, N14167, N53494, N...","[N61622, N23805]",78,240
4,12,U10045,"[N59673, N34876, N14592, N53470, N39010, N51048, N47061, N64094, N4913, N35233, N13907, N26795, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]",N21420,"[N52865, N22570, N37481, N55189, N16158, N63276, N15087, N10270, N54807, N56447, N53209, N32670,...",[N21420],53,16
...,...,...,...,...,...,...,...,...,...
85288,156959,U17467,"[N14726, N62386, N14592, N33212, N18887, N32589, N7821, N7328, N51153, N31978, N57327, N18132, N...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",N17059,"[N39813, N1746, N46868, N61681, N58871, N52536, N48592, N48282, N47173, N8331, N461, N1314, N211...",[N17059],15,39
85289,156961,U21593,"[N2235, N22975, N64037, N47652, N11378, N42515, N56211, N37204, N6837, N11930, N46917, N2960, N2...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]",N50055,"[N7432, N58559, N1954, N43353, N14343, N13008, N28833, N25114, N28833, N44495, N996, N14441, N61...",[N50055],62,31
85290,156962,U10123,"[N3841, N61571, N58813, N28213, N4428, N25796, N13579, N32118, N22457, N62128, N21309, N59385, N...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",N30212,"[N9803, N104, N24462, N57318, N55743, N40526, N31726, N54842, N26026, N56889, N2309, N41298, N63...",[N30212],32,89
85291,156963,U75630,"[N55913, N62318, N53515, N10960, N9135, N57081, N8595, N30648, N47572, N20676, N35011, N65027, N...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",N46283,"[N29898, N59704, N4408, N9803, N53644, N26103, N812, N60919, N49319, N54655, N41375, N31248, N52...",[N46283],23,43


In [13]:
def truncat_data(row, n_candidate=10, n_history=10):
    # trunct candiadte
    cache = copy.deepcopy(row.impression_list)
    cache.remove(row.pos_target)
    neg_target = cache.pop(0)
    candidate = [row.pos_target] + [neg_target] + cache[:8]
    # trunct history
    history = row.history_list[-n_history:]
    
    return neg_target, candidate, history

MIND_dev_df_fliter[["neg_target", "full_candidate", "full_history"]] = MIND_dev_df_fliter.apply(truncat_data, n_candidate=10, n_history=10, axis=1, result_type="expand")
MIND_dev_df_fliter.shape[0]
MIND_dev_df_fliter

85293

,impressionid,userid,impression_list,label_list,pos_target,history_list,impression_pos_list,history_len,impression_len,neg_target,full_candidate,full_history
0,2,U91836,"[N20678, N39317, N58114, N20495, N42977, N22407, N14592, N17059, N33677, N7821, N6890]","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]",N17059,"[N31739, N6072, N63045, N23979, N35656, N43353, N8129, N1569, N17686, N13008, N21623, N6233, N14...",[N17059],82,11,N20678,"[N17059, N20678, N39317, N58114, N20495, N42977, N22407, N14592, N33677, N7821]","[N48390, N58224, N48742, N35458, N24611, N37509, N21773, N41011, N19041, N25785]"
1,3,U73700,"[N50014, N23877, N35389, N49712, N16844, N59685, N23814, N23446, N64174, N11817, N60550, N48225,...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",N23814,"[N10732, N25792, N7563, N21087, N41087, N5445, N60384, N46616, N52500, N33164, N47289, N24233, N...",[N23814],16,36,N50014,"[N23814, N50014, N23877, N35389, N49712, N16844, N59685, N23446, N64174, N11817]","[N60384, N46616, N52500, N33164, N47289, N24233, N62058, N26378, N49475, N18870]"
2,7,U8355,"[N51346, N33848, N15132, N10688, N6342, N61359, N7809, N64397, N27079, N47149, N39844, N52585, N...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",N55204,"[N8419, N15771, N1431, N5888, N18663, N24123, N22130, N20286, N32095, N46868, N55310, N31931, N3...",[N55204],35,173,N51346,"[N55204, N51346, N33848, N15132, N10688, N6342, N61359, N7809, N64397, N27079]","[N306, N30160, N41797, N47482, N2606, N20886, N47054, N64631, N3933, N40509]"
3,11,U89744,"[N47572, N45523, N64560, N53245, N8509, N51951, N37185, N23814, N7901, N26220, N45590, N12028, N...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",N61622,"[N24422, N25287, N39121, N41777, N58226, N119, N29197, N33727, N45189, N13480, N14167, N53494, N...","[N61622, N23805]",78,240,N47572,"[N61622, N47572, N45523, N64560, N53245, N8509, N51951, N37185, N23814, N7901]","[N27023, N8959, N63658, N16710, N55911, N10888, N2735, N53428, N8519, N4654]"
4,12,U10045,"[N59673, N34876, N14592, N53470, N39010, N51048, N47061, N64094, N4913, N35233, N13907, N26795, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]",N21420,"[N52865, N22570, N37481, N55189, N16158, N63276, N15087, N10270, N54807, N56447, N53209, N32670,...",[N21420],53,16,N59673,"[N21420, N59673, N34876, N14592, N53470, N39010, N51048, N47061, N64094, N4913]","[N17463, N52227, N4745, N22929, N3933, N4607, N5019, N49990, N14137, N21428]"
...,...,...,...,...,...,...,...,...,...,...,...,...
85288,156959,U17467,"[N14726, N62386, N14592, N33212, N18887, N32589, N7821, N7328, N51153, N31978, N57327, N18132, N...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",N17059,"[N39813, N1746, N46868, N61681, N58871, N52536, N48592, N48282, N47173, N8331, N461, N1314, N211...",[N17059],15,39,N14726,"[N17059, N14726, N62386, N14592, N33212, N18887, N32589, N7821, N7328, N51153]","[N52536, N48592, N48282, N47173, N8331, N461, N1314, N21164, N90, N50872]"
85289,156961,U21593,"[N2235, N22975, N64037, N47652, N11378, N42515, N56211, N37204, N6837, N11930, N46917, N2960, N2...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]",N50055,"[N7432, N58559, N1954, N43353, N14343, N13008, N28833, N25114, N28833, N44495, N996, N14441, N61...",[N50055],62,31,N2235,"[N50055, N2235, N22975, N64037, N47652, N11378, N42515, N56211, N37204, N6837]","[N22734, N64631, N60543, N46473, N4607, N62957, N8148, N38783, N24803, N48925]"
85290,156962,U10123,"[N3841, N61571, N58813, N28213, N4428, N25796, N13579, N32118, N22457, N62128, N21309, N59385, N...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",N30212,"[N9803, N104, N24462, N57318, N55743, N40526, N31726, N54842, N26026, N56889, N2309, N41298, N63...",[N30212],3

In [14]:
MIND_dev_df_fliter_new = MIND_dev_df_fliter[:10000].reset_index(drop=True)

In [15]:
test_user_set = set(MIND_dev_df_fliter_new["userid"].values.tolist())
test_item_set = set(itertools.chain.from_iterable(MIND_dev_df_fliter_new["full_candidate"].values.tolist())) | set(itertools.chain.from_iterable(MIND_dev_df_fliter_new["full_history"].values.tolist()))
len(set(itertools.chain.from_iterable(MIND_dev_df_fliter_new["full_candidate"].values.tolist())))
len(test_user_set)
len(test_item_set)

5474

7790

15252

In [16]:
from sklearn import preprocessing

user_le = preprocessing.LabelEncoder()
user_le.fit(list(test_user_set))
print("user id unique nums:", len(user_le.classes_))

item_le = preprocessing.LabelEncoder()
item_le.fit(list(test_item_set))
print("item id unique nums:", len(item_le.classes_))

LabelEncoder()

user id unique nums: 7790


LabelEncoder()

item id unique nums: 15252


In [17]:
item_pop = item_pop[item_pop["itemid"].isin(list(set(itertools.chain.from_iterable(MIND_dev_df_fliter_new["full_candidate"].values.tolist()))))].reset_index(drop=True)
item_pop["itemid"] = pd.Series(item_le.transform(item_pop['itemid']))
item_pop

,itemid,pop
0,9599,16300
1,2314,11167
2,11925,9868
3,6728,7375
4,3383,7050
...,...,...
4546,14642,1
4547,12456,1
4548,12958,1
4549,3580,1


In [18]:
MIND_dev_df_fliter_new["userid"] = pd.Series(user_le.transform(MIND_dev_df_fliter_new['userid']))
MIND_dev_df_fliter_new["full_candidate"] = MIND_dev_df_fliter_new["full_candidate"].apply(lambda x: list(item_le.transform(x)))
MIND_dev_df_fliter_new["full_history"] = MIND_dev_df_fliter_new["full_history"].apply(lambda x: list(item_le.transform(x)))
MIND_dev_df_fliter_new["pos_id_target"] = pd.Series(item_le.transform(MIND_dev_df_fliter_new['pos_target']))
MIND_dev_df_fliter_new["neg_id_target"] = pd.Series(item_le.transform(MIND_dev_df_fliter_new['neg_target']))

MIND_dev_df_fliter_new
MIND_dev_df_fliter_new.to_csv(f"./MIND/MIND_dev_df_fliter.csv", sep="\t", index=False)

,impressionid,userid,impression_list,label_list,pos_target,history_list,impression_pos_list,history_len,impression_len,neg_target,full_candidate,full_history,pos_id_target,neg_id_target
0,2,7525,"[N20678, N39317, N58114, N20495, N42977, N22407, N14592, N17059, N33677, N7821, N6890]","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]",N17059,"[N31739, N6072, N63045, N23979, N35656, N43353, N8129, N1569, N17686, N13008, N21623, N6233, N14...",[N17059],82,11,N20678,"[1853, 2811, 7711, 12534, 2765, 8612, 3279, 1194, 6227, 14700]","[9976, 12560, 10081, 6686, 3826, 7222, 3112, 8121, 2357, 4122]",1853,2811
1,3,5854,"[N50014, N23877, N35389, N49712, N16844, N59685, N23814, N23446, N64174, N11817, N60550, N48225,...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",N23814,"[N10732, N25792, N7563, N21087, N41087, N5445, N60384, N46616, N52500, N33164, N47289, N24233, N...",[N23814],16,36,N50014,"[3629, 10402, 3642, 6669, 10319, 1802, 12924, 3536, 14101, 442]","[13108, 9478, 11090, 6089, 9673, 3735, 13551, 4290, 10251, 2314]",3629,10402
2,7,6770,"[N51346, N33848, N15132, N10688, N6342, N61359, N7809, N64397, N27079, N47149, N39844, N52585, N...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",N55204,"[N8419, N15771, N1431, N5888, N18663, N24123, N22130, N20286, N32095, N46868, N55310, N31931, N3...",[N55204],35,173,N51346,"[11790, 10773, 6266, 1351, 170, 13902, 13363, 14695, 14159, 4471]","[5401, 5289, 8322, 9720, 4202, 2865, 9608, 14221, 7715, 8010]",11790,10773
3,11,7327,"[N47572, N45523, N64560, N53245, N8509, N51951, N37185, N23814, N7901, N26220, N45590, N12028, N...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",N61622,"[N24422, N25287, N39121, N41777, N58226, N119, N29197, N33727, N45189, N13480, N14167, N53494, N...","[N61622, N23805]",78,240,N47572,"[13437, 9745, 9214, 14207, 11291, 14878, 10929, 7142, 3629, 14726]","[4458, 14991, 13964, 1772, 11965, 216, 4534, 11331, 14882, 9462]",13437,9745
4,12,4,"[N59673, N34876, N14592, N53470, N39010, N51048, N47061, N64094, N4913, N35233, N13907, N26795, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]",N21420,"[N52865, N22570, N37481, N55189, N16158, N63276, N15087, N10270, N54807, N56447, N53209, N32670,...",[N21420],53,16,N59673,"[3014, 12921, 6529, 1194, 11344, 7635, 10689, 9611, 14076, 10173]","[1970, 11010, 9710, 3406, 7715, 9342, 10462, 10390, 1085, 3015]",3014,12921
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,18305,4003,"[N44698, N6578, N45509, N29212, N51570, N10960, N1539, N48017, N9135, N23877, N36226, N23446, N4...","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",N48017,"[N29929, N26136, N1150, N28487, N13826, N55189, N56152, N28818, N51706, N30774, N307, N36751, N6...","[N48017, N45523]",56,19,N44698,"[9873, 9016, 14390, 9212, 5027, 10828, 231, 1422, 15035, 3642]","[6040, 13188, 7259, 8293, 10415, 5829, 10942, 5985, 5763, 4772]",9873,9016
9996,18306,2990,"[N59981, N42961, N33964, N23414, N154, N46029, N60872, N6099, N13259, N43502, N29128, N40725, N5...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",N23414,"[N1150, N9824, N4020, N57737, N28115, N31801, N9146, N64456, N54225, N31293, N17878]",[N23414],11,29,N59981,"[3530, 12996, 8606, 6295, 1426, 9328, 13230, 13271, 857, 8751]","[15201, 7932, 12424, 4732, 5718, 15038, 14175, 11529, 5592, 2055]",3530,12996
9997,18307,1387,"[N59267, N1228, N40109, N9639, N41387, N36226, N44698, N40905, N62318, N29212, N37338, N27737, N...","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",N40905,"[N20039, N51706, N42946, N50032, N48031, N25472, N36751, N64032, N61018, N9226, N12367, N46023, ...",[N40905],23,24,N59267,"[8102, 12806, 584, 7906, 15155, 8217, 6856, 9016, 13623, 5027]","[7922, 1773, 13370, 7510, 11748, 3817, 10980, 4788, 6371, 10081]",8102,12806
9998,18309,6630,"[N16166, N5370, N6379